In [1]:
from tensorflow.keras.applications import resnet50
import keras
import numpy as np

model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3))

c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
ROWS = 8
COLS = 96
KH_MAX = 7
XB = 1
YB = 3
KB = 1
XN = ROWS
MHZ = 250

total_clocks = 0
total_mem = 0

for i, layer in enumerate(model.layers):
    if isinstance(layer, keras.layers.convolutional.conv2d.Conv2D):

        YH, YW, CO = layer.output.shape[1:]
        KH, KW, CI, _ = layer.kernel.shape
        SH, SW = layer.strides
        XH = YH*SH
        XW = YW*SW

    elif isinstance(layer, keras.layers.core.dense.Dense):
        XH = XN
        CI, CO = layer.kernel.shape
        XW = KH = KW = 1
    
    else:
        continue
        
    L  = np.ceil(XH/ROWS)
    IT = np.ceil(CO/np.floor(COLS/KW))

    clocks = IT * (1+ XN*L*XW*(1+CI*KH))
    mem_access = \
        XB * (IT * XN * L * XW * CI * (ROWS + KH_MAX-1)) +\
        KB * (IT * CI * KH * COLS) +\
        YB * (IT * XN * L * XW * np.floor(COLS/KW) * ROWS)
    
    if layer.name in ['conv2_block1_0_conv', 'conv3_block1_0_conv', 'conv4_block1_0_conv', 'conv5_block1_0_conv']:
        continue
        total_mem += YB * (IT * XN * L * XW * np.floor(COLS/KW) * ROWS)

    total_clocks += clocks
    total_mem += mem_access


time = total_clocks/MHZ/1e6/XN
mem_access = total_mem/1024/XN/1024

print(f'clocks={total_clocks}, mb={mem_access:0f}, time_per_image={time:.3f}, fps={1/time:.2f}, mb_per_image={mem_access:.2f}, latency={XN*time:.3f}')

clocks=51317730.0, mb=96.698956, time_per_image=0.026, fps=38.97, mb_per_image=96.70, latency=0.205
